### Build a Simple LLM Application with LCEL.

This application will translate Text from English into another language.

This is a relatively simple LLM application - it's just a single LLM call plus some prompting. 

### Open AI API Key and 
### Open Source models--Llama3(Meta), Gemma2(Google), mistral--Groq

In [2]:

import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
import openai
openai.api_key= os.getenv("OPENAI_API_KEY")

groq_api_key= os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [5]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI


In [7]:
model = ChatGroq(model="gemma2-9b-it", groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002AEA31F87D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AEA31F90D0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages= [
        SystemMessage(content= "Translate the following from English to French"),
        HumanMessage(content= "Hello How are you?")
]

In [ ]:
result= model.invoke(messages)
result

AIMessage(content='Here are a few ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people) \n\n\nLet me know if you need more translations! \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 21, 'total_tokens': 103, 'completion_time': 0.149090909, 'prompt_time': 0.0013263, 'queue_time': 0.250166765, 'total_time': 0.150417209}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6307a5c2-4f8e-4679-8867-8c4971eb14e0-0', usage_metadata={'input_tokens': 21, 'output_tokens': 82, 'total_tokens': 103})

In [13]:
from langchain_core.output_parsers import StrOutputParser

parser= StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people) \n\n\nLet me know if you need more translations! \n'

### Using LCEL(LangChain Expression Language)-->> Chain the Components

In [ ]:
## This is LCEL

chain= model|parser

chain.invoke(messages)

'Here\'s how to say "Hello, How are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?**  \n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (for someone you know well)\n\n\nLet me know if you\'d like other ways to say this! \n'

# Prompt Templates

In [15]:

from langchain_core.prompts import ChatPromptTemplate

generic_template= "Translate the following into {language}:-"

prompt= ChatPromptTemplate.from_messages(
        [
            ("system", generic_template),
            ("user","{text}")
        ]
)

In [16]:
result= prompt.invoke({"language":"French", "text":"Hello How are you?"})

In [17]:
result.to_messages()

[SystemMessage(content='Translate the following into French:-', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello How are you?', additional_kwargs={}, response_metadata={})]

## Chaining together All Three Components with LCEL


In [18]:

chain=prompt|model|parser

chain.invoke({"language":"French", "text":"Hello How are you?"})

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people)\n\n\nLet me know if you\'d like more variations!\n'